# Training

In [1]:
from torch.utils.data import DataLoader
import gc
import torch
from tqdm.notebook import tqdm
import math

from detr.config import config
from augmentations.aug import get_augmentor
from detr.utils.data import WheatDataset, collate_fn
#from fasterrcnn.utils.training_handler import get_training_handler

In [2]:
conf = config()
augmentor = get_augmentor()

#conf.DATA_PATH = os.path.join('/kaggle','input', 'global-wheat-detection')
conf.BATCH_SIZE = 2
conf.DEVICE = torch.device('cuda:1') if torch.cuda.is_available() else torch.device('cpu')
#conf.SPLIT = 0.8

WD_Train = WheatDataset(conf, 
                  is_train=True, 
                  augmentation=True,
                  normalize=True,
                  augmentor=augmentor, 
                  random_seed=0)

WD_Train_Loader = DataLoader(WD_Train, batch_size=conf.BATCH_SIZE, shuffle=True, collate_fn=collate_fn)


In [5]:
WD_Train[1]

[ 988.  781. 1024.  877.] [ 26. 201. 120. 242.]
[1006.  829.   96.   36.] [ 73.  221.5  41.   94. ]
[0.9824219  0.8095703  0.09375    0.03515625] [0.07128906 0.2163086  0.04003906 0.09179688]


(array([[[-1.3234426 , -1.0517143 , -0.9877783 , ...,  0.81841505,
           0.05118249, -0.5242419 ],
         [-1.2115544 , -1.0676984 , -1.0996664 , ...,  1.0262072 ,
           0.37086272, -0.17259368],
         [-1.1476184 , -1.0836824 , -1.1476184 , ...,  1.2499834 ,
           0.83439904,  0.41881475],
         ...,
         [-0.82793814, -0.7480181 , -0.62014604, ..., -0.81195414,
          -0.7000661 , -0.7160501 ],
         [-0.7320341 , -0.49227393, -0.23652972, ..., -0.7480181 ,
          -0.588178  , -0.540226  ],
         [-0.55621   , -0.28448176,  0.0671665 , ..., -0.65211403,
          -0.49227393, -0.36440182]],
 
        [[-1.376796  , -1.0316343 , -0.91658044, ...,  1.058511  ,
           0.2147826 , -0.47554064],
         [-1.2425665 , -1.0699856 , -1.05081   , ...,  1.2502674 ,
           0.50241727, -0.09202772],
         [-1.1658639 , -1.0891613 , -1.1658639 , ...,  1.3844969 ,
           0.8859302 ,  0.44489035],
         ...,
         [-1.4151472 , -1.3192691

In [ ]:
import torch.nn.functional as F
from torch import nn

EPOCH = 0

GWD_detr_Model = torch.hub.load('facebookresearch/detr', 'detr_resnet50', pretrained=False, num_classes=1)


checkpoint = torch.hub.load_state_dict_from_url(
            url='https://dl.fbaipublicfiles.com/detr/detr-r50-e632da11.pth',
            #map_location='cpu',
            check_hash=True)
del checkpoint["model"]["class_embed.weight"]
del checkpoint["model"]["class_embed.bias"]
GWD_detr_Model.load_state_dict(checkpoint["model"], strict=False)

print(GWD_detr_Model.bbox_embed)
print(GWD_detr_Model.class_embed)

GWD_detr_Model.to(conf.DEVICE)
#print(GWD_detr_Model)
params = [p for p in GWD_detr_Model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.00001, weight_decay=0.0001)  

In [ ]:
from detr.detr.models.detr import SetCriterion
from detr.detr.models.matcher import HungarianMatcher

num_classes=1

#These are the default values in the source code
bbox_loss_coef=5.0 
giou_loss_coef=2.0
eos_coef=0.1
set_cost_class=1.0
set_cost_bbox=5.0
set_cost_giou=2.0


matcher = HungarianMatcher(cost_class=set_cost_class, cost_bbox=set_cost_bbox, cost_giou=set_cost_giou)
weight_dict = {'loss_ce': 1, 'loss_bbox': bbox_loss_coef}
weight_dict['loss_giou'] = giou_loss_coef
losses = ['labels', 'boxes', 'cardinality']

criterion = SetCriterion(num_classes, matcher=matcher, weight_dict=weight_dict,eos_coef=eos_coef, losses=losses)
criterion.to(conf.DEVICE)

In [ ]:
_iter=len(WD_Train_Loader)*(EPOCH)
clip_max_norm=0.1  #Default value in their code

_ = GWD_detr_Model.train()
_ = criterion.train()

for i in tqdm(range(EPOCH, 50, 1)):
    ep_loss_ce=0
    ep_class_error=0
    ep_loss_bbox=0 
    ep_loss_giou=0 
    ep_cardinality_error=0
    
    
    for images, targets in tqdm(WD_Train_Loader):

        images = [torch.tensor(image, dtype = torch.float32).to(conf.DEVICE) for image in images]
        targets = [{k: torch.tensor(v).to(conf.DEVICE) for k, v in target.items()} for target in targets]
        #print(targets[0]['labels'])
        
        #Main fwd pass and loss calc  
        outputs = GWD_detr_Model(images)
        
        #print(outputs['pred_logits'][0])

        loss_dict = criterion(outputs, targets)
        weight_dict = criterion.weight_dict
        summed_loss_value = sum(loss_dict[k] * weight_dict[k] for k in loss_dict.keys() if k in weight_dict)

        if math.isfinite(summed_loss_value):
            optimizer.zero_grad()
            summed_loss_value.backward()
            if clip_max_norm > 0:
                torch.nn.utils.clip_grad_norm_(GWD_detr_Model.parameters(), clip_max_norm)
            optimizer.step()
            
            ep_loss_ce+= loss_dict['loss_ce'].item()
            ep_class_error+= loss_dict['class_error'].item()
            ep_loss_bbox+= loss_dict['loss_bbox'].item()
            ep_loss_giou+= loss_dict['loss_giou'].item()
            ep_cardinality_error+= loss_dict['cardinality_error'].item()
            
            if(_iter%50 == 0):
                print("".join([k+":"+str(v.data.cpu().numpy())+", " for k,v in loss_dict.items()]))
    

        else:
            print('Loss is undefined:',summed_loss_value,'   skipping BackProp for step no:',_iter)
            print(loss_dict)
            
        _iter+=1
       
    #Divide by number of batches
    ep_loss_ce = float(ep_loss_ce/len(WD_Train_Loader))  
    ep_class_error = float(ep_class_error/len(WD_Train_Loader))
    ep_loss_bbox = float(ep_loss_bbox/len(WD_Train_Loader))
    ep_loss_giou = float(ep_loss_giou/len(WD_Train_Loader))
    ep_cardinality_error = float(ep_cardinality_error/len(WD_Train_Loader))
    _epoch_loss = ep_loss_ce+ep_class_error+ep_loss_bbox+ep_loss_giou+ep_cardinality_error
    
    print('Saving model at epoch '+str(i)+', step '+str(_iter))
    print("5 Avg Losses: {0}, {1}, {2}, {3}, {4}".format(ep_loss_ce, ep_class_error, ep_loss_bbox, ep_loss_giou, ep_cardinality_error))
    validation_score=0
    torch.save(GWD_detr_Model.state_dict(), "./GWD_DETR_SD_Epoch_{0}_Score_{1:.4f}_EpLoss_{2:.4f}.pt".format(i, validation_score, _epoch_loss))
    torch.cuda.empty_cache()
    gc.collect()


# Inference

In [24]:
import os
import torch
import torchvision

from detr.config import config
from detr.utils.inferencefns import (get_detr, 
                                     WheatInferenceDataset, 
                                     collate_fn, 
                                     format_prediction_string, 
                                     make_submission_file)
from global_utils.weighted_boxes_fusion.ensemble_boxes import nms, weighted_boxes_fusion

saved_model_dict= 'GWD_DETR_SD_Epoch_25_Score_0.0000_EpLoss_17.2091.pt'

In [2]:
conf = config()
#conf.DATA_PATH = os.path.join('/kaggle','input', 'global-wheat-detection')
conf.BATCH_SIZE = 2
conf.DEVICE = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')

WD_Inference = WheatInferenceDataset(conf, normalize=True)

In [3]:
num_classes=1
model = get_detr('resnet50', dilation=False, num_classes=num_classes)
#model.load_state_dict(torch.load('/kaggle/input/detrmodels/'+saved_model_dict, map_location=conf.DEVICE), strict=False)   #, strict=False
_ = model.eval()
_ = model.to(conf.DEVICE)

In [20]:
outputs = {idx:[] for idx in WD_Inference.IMGS}
for i in range(len(WD_Inference)):
    image_id, image = WD_Inference[i]
    results = model([torch.tensor(image).to(conf.DEVICE)])
    
    #(Xcen, Ycen, Height, Width) - scale to 0-1
    boxes = results['pred_boxes'].cpu().detach().numpy()[0]
    scores = (results['pred_logits'].cpu().detach().numpy()[0])[:,0]
    
    # changing the boxes to (x1, y1, x2, y2) - scale to 0-1
    boxes[:,0], boxes[:,1], boxes[:,2], boxes[:,3] = boxes[:,0]-boxes[:,3]/2, boxes[:,1]-boxes[:,2]/2, boxes[:,0]+boxes[:,3]/2, boxes[:,1]+boxes[:,2]/2
    
    outputs[image_id].append({'boxes':boxes, 'scores':scores, 'labels':np.ones(len(boxes))})

In [26]:
iou_thresh = 0.5
skip_box_thresh=0.1

fused_outputs = {idx:[] for idx in WD_Inference.IMGS}
for image_id in WD_Inference.IMGS:
    boxes, scores, labels = outputs[image_id][0]['boxes'], outputs[image_id][0]['scores'], outputs[image_id][0]['labels']
    boxes, scores, labels = weighted_boxes_fusion([boxes], [scores], [labels], weights=[1], iou_thr=iou_thresh, skip_box_thr=skip_box_thresh)
    
    # Now I will multiply the boxes to original scale
    fused_outputs[image_id].append({'boxes': boxes*1024, 'scores': scores, 'labels': labels})

In [28]:
strict_cut_off = 0.5
pred_strings = []
for img_id in WD_Inference.IMGS:
    try:
        stop_index = np.argwhere(fused_outputs[img_id][0]['scores']<strict_cut_off)[0][0]
    except:
        stop_index = len(fused_outputs[img_id][0]['scores'])
    
    pred_string = format_prediction_string(fused_outputs[img_id][0]['boxes'][:stop_index], scores[:stop_index])
    pred_strings.append(pred_string)
make_submission_file(WD_Inference.IMGS, pred_strings)

file_saved
